<a href="https://colab.research.google.com/github/phicaillol-cyber/repository/blob/main/ordre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================================
# URW.PA – SIGNAL QUANT + SAXO BANK EXECUTION
# ============================================================

import os
import json
import requests
import yfinance as yf
import pandas as pd
import statsmodels.api as sm
from datetime import datetime

# =========================
# PARAMÈTRES GÉNÉRAUX
# =========================

CSV_PATH = "DATASET_1000DAYS_VARIATION.csv"

FEATURES = ["GC=F", "^NDX", "AAPL"]
TARGET   = "URW.PA"

BUY_THRESHOLD  = 0.5
SELL_THRESHOLD = -0.5

MIN_TRAIN_SIZE = 60

# =========================
# SAXO BANK CONFIG
# =========================

SAXO_BASE_URL = "https://gateway.saxobank.com/openapi"

SAXO_ACCESS_TOKEN = os.getenv("SAXO_ACCESS_TOKEN")
SAXO_ACCOUNT_ID   = os.getenv("SAXO_ACCOUNT_ID")

URW_UIC    = 123456        # ⚠️ À remplacer
ASSET_TYPE = "Stock"

# =========================
# SAXO HELPERS
# =========================

def saxo_headers():
    return {
        "Authorization": f"Bearer {SAXO_ACCESS_TOKEN}",
        "Content-Type": "application/json"
    }

def get_available_cash():
    url = f"{SAXO_BASE_URL}/port/v1/accounts"
    r = requests.get(url, headers=saxo_headers())
    r.raise_for_status()
    acc = r.json()["Data"][0]
    return acc["AvailableFunds"], acc["Currency"]

def get_urw_position():
    url = f"{SAXO_BASE_URL}/port/v1/positions"
    r = requests.get(url, headers=saxo_headers())
    r.raise_for_status()

    for pos in r.json()["Data"]:
        if pos["PositionBase"]["Uic"] == URW_UIC:
            return pos["PositionBase"]["Amount"]

    return 0

def place_market_order(side, quantity):
    url = f"{SAXO_BASE_URL}/trade/v2/orders"
    payload = {
        "AccountId": SAXO_ACCOUNT_ID,
        "Uic": URW_UIC,
        "AssetType": ASSET_TYPE,
        "BuySell": side,
        "OrderType": "Market",
        "Amount": quantity,
        "ManualOrder": True
    }
    r = requests.post(url, headers=saxo_headers(), data=json.dumps(payload))
    r.raise_for_status()
    return r.json()

# =========================
# 1. CHARGER DATASET HISTORIQUE
# =========================

df = pd.read_csv(
    CSV_PATH,
    parse_dates=["date"]
).set_index("date")

df = df[FEATURES + [TARGET]].dropna()

# =========================
# 2. ENTRAÎNEMENT DU MODÈLE (lag = 1)
# =========================

X = df[FEATURES]
y = df[TARGET].shift(-1)

data = pd.concat([X, y], axis=1).dropna()
data.columns = FEATURES + ["URW_TARGET"]

X_train = sm.add_constant(data[FEATURES], has_constant="add")
y_train = data["URW_TARGET"]

model = sm.OLS(y_train, X_train).fit()

# =========================
# 3. DONNÉES LIVE (clôture US)
# =========================

tickers = FEATURES + [TARGET]

prices = yf.download(
    tickers,
    period="2d",
    interval="1d",
    auto_adjust=True,
    progress=False
)["Close"]

returns_today = prices.pct_change(fill_method=None).iloc[-1] * 100

missing = returns_today[FEATURES].isna()

if missing.any():
    print("⚠️ Marchés non clôturés :", missing[missing].index.tolist())
    predicted_return = 0.0
    signal = "KEEP"

else:
    X_today = pd.DataFrame(
        [[
            returns_today["GC=F"],
            returns_today["^NDX"],
            returns_today["AAPL"]
        ]],
        columns=FEATURES
    )

    X_today = sm.add_constant(X_today, has_constant="add")

    predicted_return = model.predict(X_today).iloc[0]

    if predicted_return > BUY_THRESHOLD:
        signal = "BUY"
    elif predicted_return < SELL_THRESHOLD:
        signal = "SELL"
    else:
        signal = "KEEP"

# =========================
# 4. SAXO BANK – CAPITAL & POSITION
# =========================

cash, currency = get_available_cash()
urw_qty = get_urw_position()

# =========================
# 5. DÉCISION FINALE
# =========================

decision = "NO_ACTION"

if signal == "BUY" and cash > 500:
    decision = "BUY"
    # place_market_order("Buy", 10)

elif signal == "SELL" and urw_qty > 0:
    decision = "SELL"
    # place_market_order("Sell", urw_qty)

# =========================
# 6. SORTIE
# =========================

print("\n==============================")
print("   SIGNAL URW.PA (J+1)")
print("==============================")
print("Date :", datetime.now().strftime("%Y-%m-%d %H:%M CET"))
print("\nVariations aujourd'hui (%)")
print(returns_today[FEATURES])
print(f"\nPrévision URW.PA J+1 : {predicted_return:.3f} %")
print("Signal modèle       :", signal)
print("Capital disponible  :", cash, currency)
print("URW.PA détenu       :", urw_qty)
print("Décision finale     :", decision)
print("==============================\n")
